### Detecting insertion attacks by heuristics

According to Frontrunner-Jones insertion attacks can be detected the following way: 
 
A transfer event is triggered on the blockchain, whenever an ERC-20 token is traded.  
An Event combines the following transactional information E = (s,r,a,c,h,i,g):
- s: sender of tokens
- r: receiver of tokens
- a: number of transferred tokens
- c: contract address of token
- h: transaction hash
- i: transaction index
- g: gas price of transaction

Iterating block by block through all transfer events and checking if there are 3 events EA1, EV, EA2 for which the folloing 6 heuristics hold:  


#### Heuristic 1

Heuristic 1: 
- sender of EA1 must be identical to sender of EV and receiver of EA2
    - sA1 = sV = rA2 
- receiver of EA1 must be identical to sender of EA2.
    - rA1 = sA2 

#### Heuristics 2
- number of tokens bought by EA1 must be similar to the umber of tokens sold by EA2 ( difference of max 1%).

#### Heuristics 3
- token contract address of EA1, EV and EA2 must be identical
    - cA1 = cV2 = cA2 

#### Heuristics 4
- transaction hashes of EA1, EV and EA2 must be dissimilar
    - hA1 != hV != hA2

#### Heuristics 5
- transaction index of EA1 must be smaller than the transaction index of EV
- transaction index of EV must be smaller than the transaction index of EA2
    - iA1 < iV < i A2   

#### Heuristics 6
- the gas price of EA1 must be larger than the gas price of EV
- the gas price of EA2 must be less or equal to gas price of EV
    - gA1 > gV >= gA2 

### Implementation of Heuristics

In [96]:
from web3 import Web3
import pandas as pd
import requests


In [97]:
web3 = Web3(Web3.HTTPProvider("https://intensive-sly-mountain.quiknode.pro/a3f5256d7f2af6541d483cce3f1d49c94c01879e/"))
print("\033[92m"+str(web3.is_connected()))

In [98]:
BLOCK_NUMBER = 5574870
TRANSFER = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef" # ERC20 "Transfer"

events = web3.eth.filter({"fromBlock": BLOCK_NUMBER, "toBlock": BLOCK_NUMBER, "topics": [TRANSFER]}).get_all_entries()

### Helper Methods

In [99]:
def get_checksum_address_from_topics_hash(topics_hash):
    return web3.to_checksum_address(topics_hash.hex().replace("0x", "")[24:64])

In [100]:
def get_amount_of_tokens_from_data_hash(data_hash):
    return int(data_hash.hex().replace("0x", "")[0:64], 16)

Create a dictionary with contract-address of token as key and events as values.

In [101]:
def get_events_by_contract_address(events):

    events_by_address = {}
    
    for event in events:
        
        token_contract_address = event["address"]
        
        if token_contract_address in events_by_address:
            events_by_address[token_contract_address].append(event)
        else:
            events_by_address[token_contract_address] = [event]
            
    return events_by_address

events_by_address = get_events_by_contract_address(events)

Create a DF of form: contract-address | transactionIndex | logIndex | transaction  

Ignore all events, where not at least 3 events include contract address (A1, V, A2)

In [102]:
def create_df_of_events(events_by_address):


    df = pd.DataFrame(columns=['contractAddress',
                               'transactionIndex',
                               'logIndex',
                               'transactionHash',
                               'wallet',
                               'sender',
                               'receiver',
                               'gasPrice',
                               'amount'])
    
    for token_contract_address in events_by_address:
    
        nr_of_transactions_with_same_coin = len(events_by_address[token_contract_address])
        
        # At least 3 transactions (A1, V, A2)
        if nr_of_transactions_with_same_coin <= 2:
            continue
            
        if token_contract_address == "0xc0829421C1d260BD3cB3E0F06cfE2D52db2cE315":
            pass
        
        if token_contract_address != "0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB":
            pass
            
        for transaction in events_by_address[token_contract_address]:
            
            transaction_hash = transaction["transactionHash"].hex()
            tx_by_hash = web3.eth.get_transaction(transaction_hash)
    
    
            
            record = {
                "contractAddress": token_contract_address,
                "transactionIndex": transaction["transactionIndex"],
                "logIndex": transaction["logIndex"],
                "transactionHash": transaction_hash,
                "wallet": tx_by_hash["from"],
                "sender": get_checksum_address_from_topics_hash(transaction["topics"][1]),
                "receiver": get_checksum_address_from_topics_hash(transaction["topics"][2]),
                "gasPrice": tx_by_hash["gasPrice"] / 10 ** 9,
                "amount": get_amount_of_tokens_from_data_hash(transaction["data"])
            }    
            new_df = pd.DataFrame([record])
            df = pd.concat([df, new_df], ignore_index=True)
    
    return df
        
df_of_events = create_df_of_events(events_by_address)
df_of_events

Get for each transactionIndex first sender/receiver and last sender/receiver

In [103]:
def create_df_grouped_by_transaction_index(df_of_events):


    df_final = pd.DataFrame(columns=['contractAddress',
                               'transactionIndex',
                               'transactionHash',
                               'wallet',
                               'first_sender',
                               'first_receiver',
                               'last_sender',
                               'last_receiver',
                               'gasPrice',
                               'amount'])
    
    unique_token_contract_addresses = df_of_events["contractAddress"].unique()
        
    df_grouped_by_contractAddress_and_transactionIndex = df_of_events.groupby(['contractAddress', 'transactionIndex'])['logIndex'].agg(['min', 'max']).reset_index()
    
    for token_contract_address in unique_token_contract_addresses:
        
        df_grouped_subset = df_grouped_by_contractAddress_and_transactionIndex[df_grouped_by_contractAddress_and_transactionIndex["contractAddress"] == token_contract_address]
        
        for index, row in df_grouped_subset.iterrows():
            
            transaction_index = row["transactionIndex"]
            min_log_index = row["min"]
            max_log_index = row["max"]
                        
            min_log_index_row = df_of_events[
                (df_of_events["contractAddress"] == token_contract_address) &
                (df_of_events["transactionIndex"] == transaction_index) &
                (df_of_events["logIndex"] == min_log_index)
            ]
            
            first_sender = min_log_index_row.iloc[0]["sender"]
            first_receiver = min_log_index_row.iloc[0]["receiver"]
            
            transaction_hash = min_log_index_row.iloc[0]["transactionHash"]
            wallet = min_log_index_row.iloc[0]["wallet"]
            gasPrice = min_log_index_row.iloc[0]["gasPrice"]
            amount = min_log_index_row.iloc[0]["amount"]
            
            
            max_log_index_row = df_of_events[
                (df_of_events["contractAddress"] == token_contract_address) &
                (df_of_events["transactionIndex"] == transaction_index) &
                (df_of_events["logIndex"] == max_log_index)
            ]
            
            last_sender = max_log_index_row.iloc[0]["sender"]
            last_receiver = max_log_index_row.iloc[0]["receiver"]
            
            record = {
                "contractAddress": token_contract_address,
                "transactionIndex": transaction_index,
                "transactionHash": transaction_hash,
                "wallet": wallet,
                "first_sender": first_sender,
                "first_receiver": first_receiver,
                "last_sender": last_sender,
                "last_receiver": last_receiver,
                "gasPrice": gasPrice,
                "amount": amount
            }    
            
            new_df = pd.DataFrame([record])
            df_final = pd.concat([df_final, new_df], ignore_index=True)
            
    return df_final

df_grouped_by_transaction_index = create_df_grouped_by_transaction_index(df_of_events)
df_grouped_by_transaction_index

#### Heuristics Implementation

##### Methods to extract profit and cost from found insertion attacks

In [104]:
def get_internal_transactions(tx_hash):
    # API endpoint
    url = 'https://api.etherscan.io/api'

    # Parameters
    params = {
        'module': 'account',
        'action': 'txlistinternal',
        'txhash': tx_hash,
        'apikey': '1PN1111XBM2W5HIQCSMQH6RA65JVYPQM1R'
    }

    try:
        # Sending GET request
        response = requests.get(url, params=params, timeout=3)
    
        # Checking if request was successful
        if response.status_code == 200:
            data = response.json()
            return data["result"]
        else:
            #print('Error occurred:', response.status_code)
            return None
        
    except requests.exceptions.Timeout:
        #print('Request did not go through: timeout occurred')
        return None

In [105]:
def get_eth_rate_at_time(timestamp):
    # CryptoCompare API endpoint for historical Ethereum price
    url = f"https://min-api.cryptocompare.com/data/pricehistorical?fsym=ETH&tsyms=USD&ts={timestamp}&extraParams=your_app_name"
    
    # Send GET request to CryptoCompare API
    response = requests.get(url)
    
    # Parse JSON response
    data = response.json()
    
    # Check if response contains data
    if 'ETH' in data and 'USD' in data['ETH']:
        eth_price_usd = data['ETH']['USD']
        return eth_price_usd
    else:
        return None

In [106]:
def get_block_timestamp(block_number):
    block = web3.eth.get_block(block_number)
    timestamp = block['timestamp']
    return timestamp

In [107]:
def get_amount_eth_spent(transaction_hash):
    
    internal_transaction = get_internal_transactions(transaction_hash)
    amount_in_wei = int(internal_transaction[0]["value"])
    return amount_in_wei / 10**18

In [108]:
def get_amount_gas_spent(transaction_hash):
    
    transaction = web3.eth.get_transaction_receipt(transaction_hash)
    gas_used = transaction["gasUsed"] 
    gas_price = transaction["effectiveGasPrice"] / 10**18

    return gas_used * gas_price

In [109]:
def calculate_cost_and_profit_in_usd(block_nr, attack_tx_1, attack_tx_2):
    
    # Get amount in ETH, which were spent to buy same coins as whale
    eth_spent = get_amount_eth_spent(attack_tx_1)
    # Get gas fees from attack 1 and attack 2 transaction
    eth_fees = get_amount_gas_spent(attack_tx_1) + get_amount_gas_spent(attack_tx_2)
    
    # Get total amount that attacker spent in attack 1 and attack 2 (for coins + fees)
    cost_eth = eth_spent + eth_fees
    
    # Get amount in ETH, which were received in attack 2 (when attacker sells coins again after victim bought)
    eth_received = get_amount_eth_spent(attack_tx_2)

    # Calculate difference of received amount minus expenses: Profit
    profit_eth = eth_received - cost_eth
    
    # Convert calculated values from ETH to USD using timestamp when block of attack transactions was executed
    timestamp = get_block_timestamp(block_nr)
    eth_rate_at_timestamp = get_eth_rate_at_time(timestamp)
    
    cost_usd_at_timestamp = eth_fees * eth_rate_at_timestamp
    
    profit_usd_at_timestamp = profit_eth * eth_rate_at_timestamp
    
    return profit_usd_at_timestamp, cost_usd_at_timestamp
    

### Heuristics Implementation

Heuristics 2: Search for transactions with similar amount (at most 1% difference)

In [110]:
def get_rows_with_similar_amounts(df):
    
    def is_similar(value1, value2):
        # Check if the absolute difference between the values is within 1% of the larger value
        diff_percentage = abs(value1 - value2) / max(value1, value2) * 100
        return diff_percentage <= 1
    
    rows_with_similar_amount = []
    
    column_name = "amount"
    
    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            value1 = df.at[i, column_name]
            value2 = df.at[j, column_name]
            if is_similar(value1, value2):
                rows_with_similar_amount.append((i, j))
    
    return rows_with_similar_amount

Search for whale transaction that lies between Attack 1 and Attack 2

In [111]:
def find_whale_transactions(atk1_idx, atk2_idx, df):
    
    atk1_sender = df.iloc[atk1_idx]["first_sender"]
    possible_whales_df = df.iloc[atk1_idx+1:atk2_idx].sort_values(by="transactionIndex", ascending=False)
    
    for i in range(len(possible_whales_df)):
        
        whale_transaction = possible_whales_df.iloc[i]
        whale_receiver = whale_transaction["first_sender"]
        
        # Heuristic 1 (part of it)
        if not atk1_sender == whale_receiver:
            continue
        
        return whale_transaction      
    
    return None

In [112]:
def get_token_name_from_contract_address(contract_address):
    
    try:
        token_contract = web3.eth.contract(address=contract_address, abi=[{"constant":True,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":False,"stateMutability":"view","type":"function"}])
        return token_contract.functions.name().call()
    except:
        return None

In [113]:
get_token_name_from_contract_address("0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB")

In [114]:
def get_attacks_for_contract_address(df, contract_address, block_nr):
    
    attack_df_by_transaction_index = pd.DataFrame()
    
    # Heuristics 3 already grouped by contract address   
    # Heuristics 5 (sorting by transaction index)
    df_contract_address = df[df["contractAddress"] == contract_address].sort_values(by="transactionIndex").reset_index(drop=True)
            
    # Heuristics 2
    rows_with_similar_amount = get_rows_with_similar_amounts(df_contract_address)
    
    for combination in rows_with_similar_amount:
        
        row1_idx = combination[0]
        row2_idx = combination[1]
        
        if row2_idx - row1_idx <= 1:
            continue
        
        row1 = df_contract_address.iloc[row1_idx]
        row2 = df_contract_address.iloc[row2_idx]
        
        # Heuristic 1 (part of it)
        if not row1["first_sender"] == row2["last_receiver"]:
            continue
        
        if not row1["first_receiver"] == row2["last_sender"]:
            continue
        
        # Heuristics 6
        if not row1["gasPrice"] > row2["gasPrice"]:
            continue
            
        whale_transaction = find_whale_transactions(row1_idx, row2_idx, df_contract_address)
        
        if whale_transaction is None:
            continue
        
        # Heuristics 4
        if not row1["transactionHash"] != whale_transaction["transactionHash"] and row2["transactionHash"] != whale_transaction["transactionHash"]:
            continue
            
        profit, cost = calculate_cost_and_profit_in_usd(block_nr, row1["transactionHash"], row2["transactionHash"])
    
        attack_record = {
            "blockNumber": block_nr,
            "first_attacker": row1["wallet"],
            "whale": whale_transaction["wallet"],
            "second_attacker": row2["wallet"],
            "first_whale_gas_price_delta": round(row1["gasPrice"] - whale_transaction["gasPrice"], 2),
            "whale_second_gas_price_delta": round(whale_transaction["gasPrice"] - row2["gasPrice"], 2),
            "profit": profit,
            "cost" : cost,
            "token_contract_address": contract_address,
            "token_name": get_token_name_from_contract_address(contract_address)
        }
        
        new_df = pd.DataFrame([attack_record])
        attack_df_by_transaction_index = pd.concat([attack_df_by_transaction_index, new_df], ignore_index=True)
        
    return attack_df_by_transaction_index

get_attacks_for_contract_address(df_grouped_by_transaction_index, "0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB", 5574870)

### Putting all together

In [115]:
TRANSFER = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef" # ERC20 "Transfer"

def get_frontrunning_attacks_of_block(block_nr):
    
    events = web3.eth.filter({"fromBlock": block_nr, "toBlock": block_nr, "topics": [TRANSFER]}).get_all_entries()
    
    events_by_address = get_events_by_contract_address(events)

    df_of_events = create_df_of_events(events_by_address)
    df_grouped_by_transaction_index = create_df_grouped_by_transaction_index(df_of_events)

    unique_token_contract_addresses = df_grouped_by_transaction_index["contractAddress"].unique()
    
    df_attacks_in_block = pd.DataFrame()

    for token_contract_address in unique_token_contract_addresses:
        df_attacks = get_attacks_for_contract_address(df_grouped_by_transaction_index, token_contract_address, block_nr)
        df_attacks_in_block = pd.concat([df_attacks_in_block, df_attacks], ignore_index=True)
    
    # Some transactions involve event-chain, which are recognized as attacks multiple times. Just keep once by removing duplicates    
    df_attacks_in_block.drop_duplicates(subset=['blockNumber', 'first_attacker', 'whale', 'second_attacker'], keep='last', inplace=True)
    
    return df_attacks_in_block

#### Test random blocks from insertion data

In [116]:
get_frontrunning_attacks_of_block(5574870)

In [117]:
get_frontrunning_attacks_of_block(5599805)

In [118]:
get_frontrunning_attacks_of_block(5599933)

In [119]:
get_frontrunning_attacks_of_block(5625057)

In [120]:
get_frontrunning_attacks_of_block(9409988)